In [1]:
import pandas as pd

robot_data = pd.read_csv('robot_dataset.csv')
robot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             493 non-null    int64  
 1   adult                  493 non-null    bool   
 2   backdrop_path          344 non-null    object 
 3   belongs_to_collection  98 non-null     object 
 4   budget                 493 non-null    int64  
 5   genres                 493 non-null    object 
 6   homepage               148 non-null    object 
 7   id                     493 non-null    int64  
 8   imdb_id                453 non-null    object 
 9   original_language      493 non-null    object 
 10  original_title         493 non-null    object 
 11  overview               488 non-null    object 
 12  popularity             493 non-null    float64
 13  poster_path            447 non-null    object 
 14  production_companies   493 non-null    object 
 15  produc

In [2]:
text_data = robot_data[['title', 'tagline', 'overview']].dropna()
text_data.head()

,title,tagline,overview
0,Finch,Take an unforgettable journey.,"On a post-apocalyptic Earth, a robot, built to..."
1,Godzilla vs. Kong,One Will Fall,"In a time when monsters walk the Earth, humani..."
2,Pacific Rim,"To Fight Monsters, We Created Monsters",A ragtag band of humans band together in the y...
3,Real Steel,"If you get one shot, make it real.",Charlie Kenton is a washed-up fighter who reti...
4,Outside the Wire,Defiant by design.,"In the near future, a drone pilot is sent into..."


In [3]:
import string
import re

remove_string = '\w*robot\w*'
#remove original keywords
text_data['overview_processed'] = text_data['overview'].map(lambda x: re.sub(r'{}'.format(remove_string), '', x))
# Remove punctuation
text_data['overview_processed'] = text_data['overview_processed'].map(lambda x: re.sub(r'[,\.!?]', '', x))
# Convert the titles to lowercase
text_data['overview_processed'] = text_data['overview_processed'].map(lambda x: x.lower())
# # Print out the first rows of papers
text_data['overview_processed'].head()

0    on a post-apocalyptic earth a  built to protec...
1    in a time when monsters walk the earth humanit...
2    a ragtag band of humans band together in the y...
3    charlie kenton is a washed-up fighter who reti...
4    in the near future a drone pilot is sent into ...
Name: overview_processed, dtype: object

In [4]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

data = text_data['overview_processed'].values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['post', 'apocalyptic', 'earth', 'built', 'protect', 'life', 'dying', 'creator', 'beloved', 'dog', 'learns', 'life', 'love', 'friendship', 'means', 'human']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1)]


In [6]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.012*"earth" + 0.008*"planet" + 0.008*"life" + 0.006*"two" + 0.005*"human" '
  '+ 0.005*"alien" + 0.004*"technology" + 0.004*"man" + 0.004*"one" + '
  '0.004*"humanity"'),
 (1,
  '0.007*"world" + 0.006*"earth" + 0.005*"one" + 0.005*"machine" + '
  '0.004*"powerful" + 0.004*"find" + 0.004*"human" + 0.004*"future" + '
  '0.004*"brain" + 0.004*"save"'),
 (2,
  '0.007*"new" + 0.006*"human" + 0.005*"earth" + 0.004*"chitti" + 0.004*"dr" + '
  '0.003*"space" + 0.003*"alien" + 0.003*"year" + 0.003*"steal" + '
  '0.003*"must"'),
 (3,
  '0.005*"father" + 0.005*"one" + 0.004*"movie" + 0.004*"city" + '
  '0.004*"children" + 0.004*"help" + 0.004*"new" + 0.004*"military" + '
  '0.004*"must" + 0.003*"secret"'),
 (4,
  '0.007*"world" + 0.006*"life" + 0.005*"human" + 0.005*"earth" + '
  '0.004*"kyron" + 0.004*"one" + 0.004*"last" + 0.004*"machines" + '
  '0.004*"must" + 0.004*"first"'),
 (5,
  '0.007*"city" + 0.007*"man" + 0.005*"time" + 0.004*"year" + 0.004*"sam" + '
  '0.004*"kung" + 0.004*"

In [7]:
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

c:\users\marti\pycharmprojects\film-data-explorer\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.000910 -0.019779       1        1  15.556085
1      0.084200 -0.062601       2        1  13.828170
7     -0.075155  0.018575       3        1  12.540353
0      0.042155  0.106097       4        1  11.915701
5     -0.103940 -0.052975       5        1   9.052797
3     -0.027783  0.056213       6        1   8.396923
6      0.008521 -0.058085       7        1   8.335419
4      0.060768 -0.021684       8        1   7.967281
9      0.001870  0.036541       9        1   6.657606
2      0.010272 -0.002301      10        1   5.749666, topic_info=       Term       Freq      Total Category  logprob  loglift
8     human  23.000000  23.000000  Default  30.0000  30.0000
36    alien  12.000000  12.000000  Default  29.0000  29.0000
128    city  15.000000  15.000000  Default  28.0000  28.0000
10     life  23.000000  23.000000  Default  27.0000  27.0000
6     earth  38.000000  38.000000  Default  26.0000  26.0000
..      ...        ...        ...      ...      ...      ...
31     sets   0.841713   4.793383  Topic10  -6.1715   1.1165
102    must   1.162305  24.782201  Topic10  -5.8488  -0.2037
10     life   1.042947  23.579596  Topic10  -5.9572  -0.2623
234     man   0.923718  17.569663  Topic10  -6.0786  -0.0895
28   planet   0.903393  18.209708  Topic10  -6.1008  -0.1475

[716 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2023      9  0.830994    abduct
720       7  0.791783     abide
312       1  0.263237  actually
312       2  0.263237  actually
312       8  0.263237  actually
...     ...       ...       ...
301       7  0.166705     young
301       8  0.083352     young
301       9  0.083352     young
301      10  0.083352     young
810       7  0.791779   zathura

[1344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 8, 1, 6, 4, 7, 5, 10, 3])